In [ ]:
import functools

import numpy as np
import tensorflow as tf

In [ ]:
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

In [ ]:
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

데이터 로드하기



In [ ]:
!head {train_file_path}

In [ ]:
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [ ]:
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=5, # Artificially small to make examples easier to show.
      label_name=LABEL_COLUMN,
      na_value="?",
      num_epochs=1,
      ignore_errors=True, 
      **kwargs)
  return dataset

raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [ ]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

In [ ]:
show_batch(raw_train_data)

In [ ]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

show_batch(temp_dataset)

In [ ]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']

temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)

show_batch(temp_dataset)#일부 칼럼만 뽑기

데이터 전처리

In [ ]:
#연속 데이터 
SELECT_COLUMNS = ['survived','age','n_siblings_spouses','parch','fare']
DEFAULTS = [0,0.0,0.0,0.0,0.0]
temp_dataset=get_dataset(train_file_path,
                         select_columns=SELECT_COLUMNS,
                         column_defaults=DEFAULTS)
show_batch(temp_dataset)

In [ ]:
example_batch, labels_batch=next(iter(temp_dataset))

In [ ]:
example_batch

In [ ]:
next(iter(temp_dataset))

In [ ]:
labels_batch

다음은 모든 열을 묶는 간단한 함수

In [ ]:
def pack(features,label):
    return tf.stack(list(features.values()),axis=-1),label

이 함수를 데이터세트의 각 요소에 적용 한다

In [ ]:
packed_dataset=temp_dataset.map(pack)

for features, labels in packed_dataset.take(1):
    print(features.numpy())
    print()
    print(labels.numpy())

혼합 데이터 유형이 있는 경우, 해당 단순 숫자 필드를 분리할 수 있다.

In [ ]:
show_batch(raw_train_data)

In [ ]:
example_batch, labels_path = next(iter(temp_dataset))

숫자 특성 목록을 선택하고 단일 열로 묶는 보다 일반적인 전처리기를 정의한다

In [ ]:
class PackNumericFeatures(object):
    def __init__(self,names):
        self.names = names
    def __call__(self,features,labels):
        numeric_features=[features.pop(name) for name in self.names]
        numeric_features=[tf.cast(feat,tf.float32) for feat in numeric_features]
        numeric_features=tf.stack(numeric_features,axis=-1)
        features['numeric']=numeric_features

        return features,labels

In [ ]:
NUMERIC_FEATURES = ['age','n_siblings_spouses','parch','fare']
packed_train_data=raw_train_data.map(PackNumericFeatures(NUMERIC_FEATURES))
packed_test_data=raw_test_data.map(PackNumericFeatures(NUMERIC_FEATURES))

In [ ]:
show_batch(packed_train_data)

In [ ]:
example_batch, labels_path = next(iter(packed_train_data))

## 데이터 정규화  
연속 데이터는 항상 정규화되어야 한다

In [ ]:
import pandas as pd
desc=pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

In [ ]:
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [ ]:
STD

In [ ]:
def normalize_numeric_data(data,mean,std):
    #Center the data
    return (data-mean)/std

이제 숫자열을 만든다

In [ ]:
normalizer = functools.partial(normalize_numeric_data,mean=MEAN,std=STD)
numeric_column=tf.feature_column.numeric_column('numeric',normalizer_fn=normalizer, shape=[len(NUMERIC_FEATURES)])
numeric_columns=[numeric_column]
numeric_column

모델을 훈련할 때 이 특성 열을 포함하여 이 숫자 데이터 블록을 선택하고 중앙에 배치합니다.

In [ ]:
example_batch['numeric']

In [ ]:
numeric_layer=tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

여기에 사용된 평균 기반 정규화를 위해서는 각 열의 평균을 미리 알아야 합니다

## 범주형 데이터


In [ ]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}

In [ ]:
CATEGORIES.items()

In [ ]:
categorical_columns= []
for feature, vocab in CATEGORIES.items():
    cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
    categorical_columns.append(tf.feature_column.indicator_column(cat_col))
    

In [ ]:
categorical_columns

In [ ]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

이것은 나중에 모델을 빌드할 때 데이터 처리 입력의 일부가 됩니다.

## 결합된 전처리 레이어  
두 개의 특성 열 모음을 추가하고 ```tf.keras.layers.DenseFeatures``` 에 전달하여 두 입력 유형을 추출하고 전처리할 입력 레이어를 만듭니다

In [ ]:
preprocessing_layer=tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [ ]:
print(preprocessing_layer(example_batch).numpy()[0])

## 모델 빌드하기 
```preprocessing_layer```를 사용하여 ```tf.keras.Sequential``` 를 빌드한다

In [ ]:
model=tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

## 훈련, 평가 및 예측하기  
이제 모델을 인스턴스화하고 훈련할 수 있습니다.


In [ ]:
train_data=packed_train_data.shuffle(500)

In [ ]:
test_data= packed_test_data

In [ ]:
model.fit(train_data, epochs=20)

In [ ]:
test_loss, test_accuracy = model.evaluate(test_data)
print('\n\nTest Loss {},Test Accuracy {}'.format(test_loss,test_accuracy))

In [ ]:
predictions = model.predict(test_data)
predictions[:10]

In [ ]:
list(test_data)[0][1][:10]

In [ ]:
# Show some results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  prediction = tf.sigmoid(prediction).numpy()
  print("Predicted survival: {:.2%}".format(prediction[0]),
        " | Actual outcome: ",
        ("SURVIVED" if bool(survived) else "DIED"))

In [ ]:
prediction = tf.sigmoid(prediction).numpy()

In [ ]:
prediction
